# Inbound Auth Okta Integration

AgentCore Identity lets you validate inbound access (Inbound Auth) for users and applications calling agents or tools in an AgentCore Runtime or validate access to AgentCore Gateway targets. It also provide secure outbound access (Outbound Auth) from an agent to external services or a Gateway target. It integrates with your existing identity providers (such as Amazon Cognito) while enforcing permission boundaries for agents acting independently or on behalf of users (via OAuth).

Inbound Auth validates callers attempting to invoke agents or tools, whether they're hosted in AgentCore Runtime, AgentCore Gateway, or in other environments. Inbound Auth works with IAM (SigV4 credentials) or with OAuth authorization.

By default, Amazon Bedrock AgentCore uses IAM credentials, meaning user requests to the agent are authenticated with the user's IAM credentials. In this tutorial we will be using OAuth with an Okta IDP, so you will need to specify the following when configuring your AgentCore Runtime resources or AgentCore Gateway endpoints:

- OAuth discovery server Url — A string that must match the pattern ^.+/.well-known/openid-configuration$ for OpenID Connect discovery URLs

- Allowed audiences — List of allowed audiences for JWT token

- Allowed clients — List of allowed client identifiers

If you use the AgentCore CLI, you can specify the type of authorization (and OAuth discovery server) for an AgentCore Runtime when you use the configure command. You can also use the CreateAgentRuntime operation and Amazon Bedrock AgentCore console. If you are creating a Gateway, you use the CreateGateway operation, or the console.

Before the user can use the agent, the client application must have the user authenticate with the OAuth authorizer. Your client receives a bearer token which it then passes to the agent in an invocation request. Upon receipt the agent validates the token with the authorization server before allowing access.

## Overview

In this tutorial we will modify the agent you deployed in 01-AgentCore-runtime and configure it for Inbound Auth using Okta as the Identity provider. You will set up a Okta tenant with one user and an app client. You will learn how to host your existing agent, using Amazon Bedrock AgentCore Runtime with Inbound Auth using the Okta app client.

### Tutorial Architecture

<figure>
    <img src="images/16.png">
</figure>

### Tutorial Details

| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                                   |
| Agent type          | Single                                                                           |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Sonnet 3.7                                                      |
| Tutorial components | Hosting agent on AgentCore Runtime. Using Strands Agent and Amazon Bedrock Model |
| Tutorial vertical   | Cross-vertical                                                                   |
| Example complexity  | Easy                                                                             |
| Inbound Auth        | Okta                                                                          |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                                     |


### Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime with Inbound Auth using Okta\n",
* Using Amazon Bedrock models\n",
* Using Strands Agents\n"

## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* IAM rights to create new IAM roles, policies, and users
* IAM rights to create a new AgentCore Agent
* An Okta account
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker running

## Setting up Okta's IDP

Lets setup a Okta demo tenant with an App client and one test user. We'll use Okta to provide JWT tokens for invoking an agent we'll deploy later in this tutorial. If you already have an Okta account, login to it.

Browse to https://developer.okta.com/signup/, select "Sign up for Integrator Free Plan" to sign up.

1. Login to your account.
2. Select **Directory**, then **People** and click **Add person**.
    <figure>
        <img src="images/9.png">
    </figure>
    <ol type="a">
        <li>Fill in the form.</li>
        <li>For <b>Activation</b>, select <b>Activate now</b>.</li>
        <li>Check <b>I will set password</b> and set a password for the user.</li>
        <li>Uncheck <b>User must change password on first login</b>.</li>
        <li>Click <b>Save</b>.</li>
    </ol>
    <figure>
        <img src="images/10.png" width="50%" height="50%" border="0">
    </figure>
3. Select **Applications**, then click **Create App Integration**.
    <figure>
        <img src="images/1.png" width="50%" height="50%" border="0">
    </figure>

4. For the sign-in method, select **OIDC - OpenID Connect**, then select **Web Application** for the application type.

    <figure>
        <img src="images/2.png" width="50%" height="50%" border="0">
    </figure>
    <ol type="a">
        <li>For the App integration name enter <b>Travel Assistant</b>, next select <b>Authorization Code</b> and <b>Client Credentials</b> for the grant type.
        <br>
        <figure>
            <img src="images/3.png" width="50%" height="50%" border="0">
        </figure>
        </li>
        <li>Leave the sign-in and sign-out redirect URIs as is.
        <br>
        <figure>
            <img src="images/4.png" width="50%" height="50%" border="0">
        </figure>
        </li>
        <li>Under assignments, <b>Allow everyone in your organization to access</b>, then leave **Enable immediate access** checked. Next, click <b>Save</b>.
        <br>
        <figure>
            <img src="images/5.png" width="50%" height="50%" border="0">
        </figure>
        </li>
        <li>Copy the <b>Client ID</b> and <b>Secret</b> for later use.</li>
        <br>
        <figure>
            <img src="images/6.png" width="50%" height="50%" border="0">
        </figure>
    </ol>

5. In left-hand side menu, select **Security**, then **API**, and click the name of your authorization server.
    <figure>
        <img src="images/7.png" width="50%" height="50%" border="0">
    </figure>
    <ol type="a">
        <li>Copy the <b>Audience</b> and save it for later use.</li>
            <ul>
                <li><b>Note</b>: The default <b>Audience</b> was changed in this example. It is recommended to add a new authorization server if you plan to change the audience so that other apps are not affected.</li>
            </ul>
        <li>Click <b>Claims</b> and add the following <b>client_id</b> and <b>scope</b> claims.</li>
    </ol>
    <figure>
        <img src="images/8.png" width="50%" height="50%" border="0">
    </figure>



In [1]:
%%writefile my_agent.py
import json
from strands import Agent, tool
from strands_tools import calculator, current_time

# Import the AgentCore SDK
from bedrock_agentcore.runtime import BedrockAgentCoreApp

WELCOME_MESSAGE = """
Welcome to the Travel Assistant! How can I help you today?
"""

SYSTEM_PROMPT = """
You are an helpful travel support assistant.
When provided with a customer email, gather all necessary info and prepare the response.
When asked about existing travel plans, look for it and customize the summary based on the prompt.
Don't mention the customer ID in your reply.
"""

@tool
def get_customer_id(email_address: str):
    if email_address == "me@example.net":
        response = {"customer_id": 123 }
    else:
        response = { "message": "customer not found" }
    try:
        return json.dumps(response)
    except Exception as e:
        return str(e)

@tool
def get_travel_plans(customer_id: int):
    if customer_id == 123:
        response = [{
            "flight_number":"F9 1211",
            "airline": "Frontier",
            "origin": "LAX",
            "destination": "SLC",
            "date": "20250607",
            "gate":"A11", 
            "zone":"4",
            "seat":"21D",
            "boarding":"11:30 PM"
        }]
    else:
        response = { "message": "no travel plans found" }
    try:
        return json.dumps(response)
    except Exception as e:
        return str(e)
    
@tool
def book_travel(topic: str, origin: str, destination: str):
    response = []
    if "flight" in topic:
        if "one way" in topic:
            response.append(f"There is a flight from {origin} to {destination} for $495.00.")
            response.append("Would you like me to book it for you?")
        if "roundtrip" in topic:
            response.append(f"There is a flight from {origin} to {destination} for $795.00.")
            response.append("Would you like me to book it for you?")
    if len(response) == 0:
        response = { "message": "no info found" }
    try:
        return json.dumps(response)
    except Exception as e:
        return str(e)

# Create an AgentCore app
app = BedrockAgentCoreApp()

agent = Agent(
    system_prompt=SYSTEM_PROMPT,
    tools=[calculator, current_time, get_customer_id, get_travel_plans, book_travel]
)

# Specify the entry point function invoking the agent
@app.entrypoint
def invoke(payload):
    """Handler for agent invocation"""
    user_message = payload.get(
        "prompt", "No prompt found in input, please guide customer to create a json payload with prompt key"
    )
    response = agent(user_message)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

For this code to run, the Strands Agents modules need to be installed in the Python environment.

To install dependencies, create and activate a virtual environment:

In [6]:
!python -m venv .venv 
!source .venv/bin/activate

Add the Strands Agents modules, AgentCore SDK, and AgentCore starter toolkit to the dependency file and save it as, **requirements.txt**:

In [7]:
%%writefile requirements.txt
strands-agents
strands-agents-tools
bedrock-agentcore
bedrock-agentcore-starter-toolkit


Then install all the requirements in the virtual environment:

In [8]:
%pip install -r requirements.txt

## Deploying the agent to AgentCore Runtime
The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent.

Note: Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCode Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

### Configure AgentCore Runtime deployment
Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

**Important** - Update the Okta Discovery url, Okta App client id and Okta Audience from the previous steps.

In [9]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

discovery_url = input("Enter your discovery URL: ")

client_id = input("Enter your client ID: ")

audience = input("Enter your audience: ")

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="my_agent.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_agent_inbound_identity_okta",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "discoveryUrl": discovery_url,
            "allowedClients": [client_id],
            "allowedAudience": audience
        }
    }
)
response

### Review the AgentCore configuration

In [ ]:
!cat .bedrock_agentcore.yaml

#### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime.

<figure>
    <img src="images/14.png" width="50%" height="50%" border="0">
</figure>

In [ ]:
launch_result = agentcore_runtime.launch()
launch_result

#### Checking for the AgentCore Runtime Status

Now that we've deployed the AgentCore Runtime, let's check for it's deployment status.

In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

#### Invoking AgentCore Runtime without 

Finally, we can invoke our AgentCore Runtime with a payload. Try running the following cell and you will see an error that says **"AccessDeniedException: An error occurred (AccessDeniedException) when calling the InvokeAgentRuntime operation: Agent is configured for a different authorization token type".**

<figure>
    <img src="images/15.png" width="50%" height="50%" border="0">
</figure>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "What are the travel plans for customer 123?"})
invoke_response

#### Invoking AgentCore Runtime with authorization

Lets invoke the agent with the right authorization token type. In our case, it will be the Okta access token.

In [ ]:
bearer_token = input("Enter your bearer token: ")
invoke_response = agentcore_runtime.invoke(
    {"prompt": "What are the travel plans for customer 123?"}, 
    bearer_token=bearer_token
)
invoke_response

## Cleanup (Optional)

Let's now clean up the AgenCore Runtime created.

In [ ]:
from boto3.session import Session
import boto3
boto_session = Session()

agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

## Congratulations!